# Cluster Analysis

Here, no real model involved, not super-involved or difficult.
This kind of analysis can work nicely with MDS, where we do cluster analysis, and use MDS to visualize our data, drawing circles where the clusters were found, and it's a nice visual representation.

That said, recall that MDS looked for dissimilarity, and finding ways to put objects far apart.
In cluster analysis, we're looking at similarity, and trying to distinguish sets of related objects.

## Basic approach

Assume we measure $p$ attributes on $n$ objects.
Like MDS, we can construct an $n \times n$ matrix of distances between objects, based on our attributes.

## Metric distance measures.

Now, there's a variety of ways we could define similarity/distance/proximity.
One way to distinguish this is between *metric* and *non-metric* measures of distance (different from meaning of "metric" in MDS).

definition: A distance measure is called *metric* if the following properties hold:
1. **symmetry**: $\forall x, y; d(x, y) = d(y, x)$
2. **triangle inequality**: $\forall x, y, z; d(x, y) \le d(x, z) + d(z, y)$
3. **distinguishability of nonidenticals**: $\forall x, y; d(x, y) \ne 0 \iff x \ne y$
4. **indistinguishability of identicals**: $\forall x, y; d(x, y) = 0 \iff x = y$.

### Examples of distance measures

1. Distance-based measures  
    - **Euclidean distance**: $d_{ij} = \left[ \sum_{k=1}^p (X_{ik} - X_{jk})^2 \right]^{\frac{1}{2}}$
    - **Manhattan/"City-block" distance**: $d_{ij} = \left( \sum_{k=1}^p |X_{ik} - X_{jk}| \right)$
    - **Minkowski metric** (generalized distance): $d_{ij} = \left[ \sum_{k=1}^p (X_{ik} - X_{jk})^r \right]^{\frac{1}{r}}$  
    **Note**, for large differences in a single variable, a larger $r$ will make the overall distance much larger.
    Basically, a large $r$ emphasizes differences in individual attributes, while smaller $r$ emphasizes objects having differences in many attributes.
    - **Mahalanobis distance**: Let $X_i$ be a column vector of object $i$'s attributes, and let $S$ be the pooled within-group covariance matrix. Then $d_{ij} = (X_i - X_j)' S^{-1} (X_i - X_j)$  
    **Note**, the covariance matrix tells us if two variables are very correlated, and I'm not quite sure what inverse of that matrix means, but basically the use in this calculation will reduce the weight on highly-correlated variables. That way, a moderate change in two strongly-correlated variables only "counts" as one moderate change, basically.
2. Matching or association-type measures
    These are used when variables are "nominally scaled," i.e. there is no quantitative ordering of the scores on each var.
    e.g. all our $p$ variables might be booleans indicating an object has an attribute or does not have it.
    Suppose, for example, we have people as objects, and variables are cities.
    Each is a bool indicating whether the person visited the given city.
    To create a similarity measure of cities, we could compute frequency of (a) # people who visited both, (b) # people who visited A but not B, (c) # people who visited B but not A, and (d) # people who visited neither.
    Some specific measures are:  
    - **Simple matching coefficient**: $d_{AB} = \frac{a + d}{a + b + c +d}$
    - **Jaccard's coefficient**: $d_{AB} = \frac{a}{a + b + c}$  
    **Note** in this case, we're saying that $d$ doesn't mean much, because if $d$ is large, it just means nobody visits these two cities, which doesn't really tell us much about similarity of the two.
    - **Modified Jaccard coefficient**: $d_{AB} = \frac{2a}{2a + b + c}$  
    **Note** this was just professor's suggestion, idea is to give greater weight to the "both visited" case, since there's only one of these and two of the "visited one" cases.
3. Correlation measures
    While correlation is a measure of similarity between objects, it does not have the properties to be a metric measure.
    That's not very nice.
    In addition, it throws away means and standard deviations, generally ignores scales.
    We could thus get two very different objects that look more similar than they are, or something.
    Like, two cars that are twice as fast as they are efficient would appear similar, even if one was both "faster" and "more efficient" than the other.

## Clustering algorithms

The design of a clustering algorithm needs to decide how to decide on distance between two objects, as well as how to represent locations of clusters and distances between clusters.
Nearest neighbor is a common way to do this, where distance between clusters is defined as the smallest distance between any pair of elements in the clusters,  
i.e. $d(A, B) = \min\limits_{a \in A, b \in B} d(a, b)$.  
Broadly, distance between clusters can be done through single-linkage (e.g. nearest neighbor), complete-linkage, or average-linkage.
There's a couple classes of algos. *Hierarchical* and *non-hierarchical*.

1. Hierarchical clustering
  Idea is that we start with each object as own cluster, and apply "merge" steps until we reach a "good" solution.
  Each algorithm in this class is defined by its own way of choosing which clusters should be merged, or by its stopping criteria.
  Technically, these are *glomerative hierarchical clustering* approaches.  
  We could also take opposite approach, and start with all in one cluster and then subdivide them.
  However, in practice, most hierarchical clustering algorithms are glomerative.


## Glomerative Hierarchical Clustering

### Distances between clusters:
1. Single-linkage: Defined by a single pair of elements, one from each cluster. I think this tends to let us see outliers very separate from the other clusters in the way clusters get constructed. The outliers tend to not join a cluster until a merging step.
    - Nearest-neighbor: described above
2. Complete-linkage: Apparently, this tends to form "globular" (as in globe-like) clusters; maybe it tended to pair things off, then combine pairs, etc.? In the end, gives more homogeneous clusters, or something.
    - Farthest-neighbor: Opposite of nearest-neighbor, where now we find the most distant pair of elements with one in each cluster.
3. Average-linkage: Again, doesn't seem like we are looking at any specific instances of this. I guess it's just sort of halfway between the patterns of the other two. Gives some homogeneity, but still leaves outliers out on their own.
4. Centroid Method: Instead of basing distances on pairs of points, we go back to the original data and define new measure of distance. In this case, we take a cluster, calculate the centroid of the cluster (mean point), and treat this as a cluster location. Then we define distance between two clusters as the distance between their centroids.
This is a bit quicker and simpler than an average-linkage approach, but doens't really account for variability (or geometric 'size') within the cluster. Also, something about lacking "invariance" if we do a monotonic transform on the similarity measure.
5. Ward's error sum of squares method: This sounds like an algorithm, rather than a distance... Anyway, idea is to minimize variance of elements within each cluster.
That is, on the merge step, we calculate the SSE for each pair of existing clusters, and merge the pair with smallest SSE.
In our example, this 'quickly' groups nearby clusters, with respect to the distance metric, then has a 'slow' mixing of the bigger clusters.

#### Invariance with respect to monotonic transforms

There are occasions where we might decide we don't just want to use, say, Euclidean distance.
For example, there could be a case where we have some data where we want to use log-Euclidean distance.  
This is a monotonic transform on our distance metric.
It may be nice to know that our clustering algorithm will yield the same results regardless of whether we (monotonically) transform our distances or not.

Clearly, single- and complete-linkage approaches will always give same results, because for a monotonic function, clearly the smallest distance remains smallest (or tied for smallest) and largest remains largest (or tied for largest), by definition of monotonic function.

### Considerations for choosing a method:
- If we want clusters to be relativley spherical, with approximately equal variance/size, prefer Ward's method or average-linkage
- Single-linkage gives relatively elongated clusters with different variances/sizes.
- For globular (spherical?) clusters where variance/size differs, prefer complete-linkage.
- Space-contracting vs. space-dilating is one way to characterize these approaches. I guess this is more for how we choose to do distance. So Manhattan distance tends to be space-contracting, where distance on any one dimension doesn't magnify overall distance, where other distance measures may be "space-dilating" and magnify distance when one dimension has a large difference.

## Divisive Hierarchical Clustering

We usually want to have a relativelly small number of clusters.
When we do merging/glomerative approaches, we end up with some elements that get merged early, and any solution with any (small) number of clusters must have those elements together.
Thus, it can sometimes be nice to start with all elements in one cluster to start, and then perform only a few subdivision steps.

## Clustering Columns

We typically do clustering of rows to determine which objects of interest are most closely related.
However, it's perfectly fine to apply our clustering algorithm to the columns.
I suppose interpretation changes a bit, but could tell us which columns are closely related, in some sense?

## Choosing a stopping point

It can be a challenge to determine when we should stop our clustering.
Obviously, our approaches above for glomerative clustering typically allow for us to go all the way down to one cluster.
But what should the stopping point be?
There are a few indices used to do this:

1. Crap, moved on from this part of notes already... there were 3 options listed though.
2. This was something about taking sum-of-squares before and after merging a pair of clusters, then dividing difference by total sum-of-squares, and this gives us sense of how quickly we lose explanatory power with a merger.
3. N/A

## K-Means Clustering

One of the limitations we noted with glomerative clustering is that if two elements are put together in a cluster early on, then we can't get any smaller number of clusters where those two are apart.
Bascially, any earlier step forces future steps to be based on earlier clusters.  
K-means tries to address this, but requires us to say exactly how many clusters to include.
In this way, it's a bit more of a confirmatory than exploratory approach.

In general, it performs an iterative search, and looks for convergence.
One disadvantage is that it may converge to different solutions based on different initial conditions.

## Partitioning/Non-Hierarchical Clustering

This is the other approach to clustering we have available.
Instead of merging or subdividing clusters in a way that creates a hierarchy, we can take an approach that shuffles around objects within a set number of clusters (or partitions) until we obtain a "best" solution.

I'm not really clear if k-means is another term for partitioning/non-hierarchical, of if it's really its own instance of those.

In general, these approaches use the following steps:

1. Start by partitioning our sets into $k$ clusters, can be done randomly.
2. Compute centroid of each of the $k$ clusters.
3. Given the set of centroids, place each object in the "best fit" cluster, e.g. cluster with nearest centroid.
4. Compute new centroids after the repartitioning in step 3.
5. Repeat from step 3, stopping when step 3 does not result in a change in partitions.

Different partitioning approaches typically vary in two spots:

1. Step 1 (choosing initial clusters).
    That is, we don't have to do this randomly.
2. Step 3 (how elements are re-assigned).
    That is, we don't have to use nearest centroid.

### Choices of initial clusters

* Define initial cluster centroids as locations of first $k$ objects, assign all objects based on those centroids.  
    A potential negative is that we get a bunch of cluster centroids all near each other.
    That's not ideal.
* Put one centroid at location of first object, and remaining centroids at the next $k-1$ objects whose distances are pairwise greater than a selected minimum distance.  
    This helps us avoid initial clusters all near each other.
* Choose $k$ objects at random as initial centroids
* Pick the $k$ objects that are farthest apart.

### Choices of reassignment criteria

* Given computed centroids, reassign all objects to nearest centroid.  
    Doesn't recompute centroids until all objects have been clustered, as in general algorithm above.
* **Running means** approach: Merge steps 3 and 4, recomputing centroids after each object gets reassigned.
* Reassign to new clusters based on some other statistical measure, e.g. sum-of-squares

### Other notes:

For SPSS output from a clustering algorithm, we get an ANOVA table with F-scores and p-values for the clustering mean squares and error mean squares.
So, cluster mean square is based on sum of squared distances from grand mean of data, or something.
Like, on each dimension, we've got data in clusters, we can take the mean within the cluster, that's the cluster centroid location on that dimension.
Then we take the sum of squared distances between centroids and overall average for all data, to get cluster mean squares, I think.
And then mean square error is... maybe the sum of squared distances of each object from its cluster centroid? Or something else?
Fuck, I wish I knew how any of this bullshit was calculated for a normal ANOVA table.
I really should have gone to lectures last semester when I was enrolled in stuff that (a) actually covered this, unlike this course, and (b) didn't have a horrifically terrible lecturer, unlike 761...

However, it's worth noting that statistical significance is not meaningful here in an inferential statistics way.
Specifically, there is no independence between cluster numbering and the data itself.
It's as dependent as you can get; clusters are chosen specifically based on the already-observed data.

Apparently, for example on board, we're using standardized z-scores.
So variance is 1 for each variable.
We have 11 objects.
Then I guess this works out so that the total sum of squares for each variable is 10, something about losing one degree of freedom bringing us down to 10.
Fuck, I wish I had any algebraic intuition for any of this at all.
Fucking need to just take professor's word on it because I'm fucking stupid.
Anyway, then we have 5 variables, so total sum of squares across everything is 50.
Um, not really clear where we're going with this.
But then if we take sum down the cluster mean square column, that's our SSC, and sum down the mean square error column, we get SSE, and then our $R^2$ is just $\frac{SSC}{SSE}$.
Fucking fuck I know fucking nothing about anything. 
Zero fucking intuition for why we get something that seems like it should be objectively fucking obvious. Fuck me.
So where the fuck does the 50 show up here?
In the example, our SSC was about 23, and the SSE was around 2.
Where the fuck did the other 25 go?
If I knew fucking anything, I'd probably get it. Fuck this. Fuck me.

Wait, actually, $R^2 = \frac{SSC}{SST}$, I'm an idiot.
It's the $F$ that is $\frac{SSC}{SSE}$. 
So the knowledge that the total sum-of-squares is 50 was useful.
I'm just that dumb.

## Using R^2 to choose number of clusters

In general, if we increase the number of clusters in our solution, we get a larger $R^2$.
This is because more clusters gives us more detailed description of data, better able to account for variance.
We can just compute a solution for several values of $k$, and graph $R^2$ on y-axis vs. increasing $k$ on x-axis.
From that, we can interpret similar to a scree plot (though this is increasing, rather than decreasing), and choose our best option for $k$.